In [5]:
import pandas as pd
patients1 = pd.read_csv('C:\\Users\\Mahi2\\capstone\\capstone_healthpredictors\\data\\patients.csv')
# View a few rows (table)
print(patients1.head().to_string())

                                     Id   BIRTHDATE DEATHDATE          SSN    DRIVERS    PASSPORT PREFIX        FIRST        LAST SUFFIX MAIDEN MARITAL   RACE    ETHNICITY GENDER                      BIRTHPLACE                         ADDRESS         CITY          STATE           COUNTY     ZIP        LAT        LON  HEALTHCARE_EXPENSES  HEALTHCARE_COVERAGE
0  f0f3bc8d-ef38-49ce-a2bd-dfdda982b271  2017-08-24       NaN  999-68-6630        NaN         NaN    NaN   Jacinto644     Kris249    NaN    NaN     NaN  white  nonhispanic      M      Beverly  Massachusetts  US       888 Hickle Ferry Suite 38  Springfield  Massachusetts   Hampden County  1106.0  42.151961 -72.598959              8446.49              1499.08
1  067318a4-db8f-447f-8b6e-f2f61e9baaa5  2016-08-01       NaN  999-15-5895        NaN         NaN    NaN      Alva958  Krajcik437    NaN    NaN     NaN  white  nonhispanic      F       Boston  Massachusetts  US             1048 Skiles Trailer      Walpole  Massachusetts   Norfolk

In [6]:
print("\nShape (rows, columns):", patients1.shape)
print(patients1.columns.tolist())



Shape (rows, columns): (12352, 25)
['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']


In [8]:
print(patients1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12352 entries, 0 to 12351
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   12352 non-null  object 
 1   BIRTHDATE            12352 non-null  object 
 2   DEATHDATE            2352 non-null   object 
 3   SSN                  12352 non-null  object 
 4   DRIVERS              10399 non-null  object 
 5   PASSPORT             9845 non-null   object 
 6   PREFIX               10110 non-null  object 
 7   FIRST                12352 non-null  object 
 8   LAST                 12352 non-null  object 
 9   SUFFIX               124 non-null    object 
 10  MAIDEN               3540 non-null   object 
 11  MARITAL              8833 non-null   object 
 12  RACE                 12352 non-null  object 
 13  ETHNICITY            12352 non-null  object 
 14  GENDER               12352 non-null  object 
 15  BIRTHPLACE           12352 non-null 

In [9]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
print(patients1.describe())

          ZIP      LAT      LON  HEALTHCARE_EXPENSES  HEALTHCARE_COVERAGE
count 6700.00 12352.00 12352.00             12352.00             12352.00
mean  1991.14    42.27   -71.36            823340.16             13143.89
std    446.36     0.28     0.61            560901.64             43923.34
min   1001.00    41.24   -73.48               500.00                 0.00
25%   1757.00    42.12   -71.56            271270.82              3228.08
50%   2116.00    42.31   -71.16            822483.43              6138.94
75%   2199.00    42.45   -71.03           1359019.21             11442.49
max   2861.00    42.90   -69.92           3178011.87           2199825.08


In [10]:
print(patients1.isnull().sum())

Id                         0
BIRTHDATE                  0
DEATHDATE              10000
SSN                        0
DRIVERS                 1953
PASSPORT                2507
PREFIX                  2242
FIRST                      0
LAST                       0
SUFFIX                 12228
MAIDEN                  8812
MARITAL                 3519
RACE                       0
ETHNICITY                  0
GENDER                     0
BIRTHPLACE                 0
ADDRESS                    0
CITY                       0
STATE                      0
COUNTY                     0
ZIP                     5652
LAT                        0
LON                        0
HEALTHCARE_EXPENSES        0
HEALTHCARE_COVERAGE        0
dtype: int64


In [15]:
patients1['memberid'] = patients1['Id'].str[:8]
# Check the result
print(patients1.head().to_string())

unique_memberids = patients1['memberid'].nunique()
print(unique_memberids)

                                     Id   BIRTHDATE DEATHDATE          SSN    DRIVERS    PASSPORT PREFIX        FIRST        LAST SUFFIX MAIDEN MARITAL   RACE    ETHNICITY GENDER                      BIRTHPLACE                         ADDRESS         CITY          STATE           COUNTY     ZIP   LAT    LON  HEALTHCARE_EXPENSES  HEALTHCARE_COVERAGE  memberid
0  f0f3bc8d-ef38-49ce-a2bd-dfdda982b271  2017-08-24       NaN  999-68-6630        NaN         NaN    NaN   Jacinto644     Kris249    NaN    NaN     NaN  white  nonhispanic      M      Beverly  Massachusetts  US       888 Hickle Ferry Suite 38  Springfield  Massachusetts   Hampden County 1106.00 42.15 -72.60              8446.49              1499.08  f0f3bc8d
1  067318a4-db8f-447f-8b6e-f2f61e9baaa5  2016-08-01       NaN  999-15-5895        NaN         NaN    NaN      Alva958  Krajcik437    NaN    NaN     NaN  white  nonhispanic      F       Boston  Massachusetts  US             1048 Skiles Trailer      Walpole  Massachusetts   Norfo

In [12]:
patients1['COUNTY'].unique()

array(['Hampden County', 'Norfolk County', 'Plymouth County',
       'Suffolk County', 'Franklin County', 'Middlesex County',
       'Bristol County', 'Essex County', 'Worcester County',
       'Hampshire County', 'Barnstable County', 'Berkshire County',
       'Dukes County', 'Nantucket County'], dtype=object)

In [13]:
patients1[['HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']].describe()

,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
count,12352.00,12352.00
mean,823340.16,13143.89
std,560901.64,43923.34
min,500.00,0.00
25%,271270.82,3228.08
50%,822483.43,6138.94
75%,1359019.21,11442.49
max,3178011.87,2199825.08
